<a href="https://colab.research.google.com/github/asmaa158/healthcare-dataset-stroke-data/blob/main/DataSpam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import string
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import WhitespaceTokenizer
pd.set_option('display.max_colwidth',1000)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
#Load the data
from google.colab import files
uploaded = files.upload()

Saving data_spam.csv to data_spam (2).csv


In [20]:
#Read Csv file
data = pd.read_csv('data_spam.csv',encoding='latin-1')

In [21]:
data.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], axis=1,inplace=True)
data.rename(columns={'v1': 'Class', 'v2': 'Text'}, inplace=True)
data['Class'] = data['Class'].map( {'spam': 1, 'ham': 0})


In [22]:
# preprocessing---------------------------------------------------
#lowerCase letters
data['Text'] = data['Text'].apply(lambda x: x.lower())

#remove_punctuation
def remove_punctuation(text):
    punctuationfree = ''.join([i for i in text if i not in string.punctuation])
    return punctuationfree


data['Text2'] = data['Text'].apply(lambda x: remove_punctuation(x))
# tokenization
def tokenization(text):
    tk = WhitespaceTokenizer()
    return tk.tokenize(text)

data['Text2'] = data['Text2'].apply(lambda x: tokenization(x))
# remove stopwords

stopwords = stopwords.words('english')


def remove_stopwords(text):
    output = [i for i in text if i not in stopwords]
    return output
data['Text2'] = data['Text2'].apply(lambda x: remove_stopwords(x))

# stemming
#porter_stemmer = PorterStemmer()


#def stemming(text):
  #  stem_text = [porter_stemmer.stem(word) for word in text]
    #return stem_text


#data['Text2'] = data['Text2'].apply(lambda x: stemming(x))

# Lemmatization

lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    lemm_text = [lemmatizer.lemmatize(word) for word in text]
    return lemm_text

data['Text2'] = data['Text2'].apply(lambda x: lemmatize_words(x))

# End preprocessing---------------------------------------------------data

In [23]:
#Bag of words---------------------------------------------------    
from sklearn.feature_extraction.text import CountVectorizer
dataSample=data[0:10]
text = data['Text2'].map(' '.join)
cv = CountVectorizer()
cv2 = cv.fit_transform(text)

dataFrame=pd.DataFrame(cv2.toarray(),columns=cv.get_feature_names_out())
#End of Bag of words---------------------------------------------------  
dataFrame

,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,ìï,ìïll,ûthanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Splitting training data into train 70% and test 30%

df_train = dataFrame.iloc[:4457]
df_test = dataFrame.iloc[4457:]


df_train['Class'] = data['Class']

Y = df_train['Class']
X = df_train.drop('Class',axis=1)

from sklearn import model_selection
X_train,X_test,y_train,y_test = model_selection.train_test_split(X,Y,test_size=0.3, random_state=0)
#End of Splitting training data into train 70% and test 30%


<ipython-input-24-cf1e752ece8c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Class'] = data['Class']


In [25]:
#classification
#Naive bayes classifier
print('*****************Naive bayes classifier*******************')
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score

gnb = GaussianNB()

mnb = MultinomialNB()

bnb = BernoulliNB()

gnb.fit(X_train,y_train)


*****************Naive bayes classifier*******************


GaussianNB()

In [26]:
#Evaluating
y_pred1 = gnb.predict(X_test)
#accuracy
print('*******************GaussianNB*****************')
print('Accuracy score is',accuracy_score(y_test,y_pred1))

# F1-SCORE
f1=f1_score(y_pred1,y_test)
f1_round=round(f1,2)
print('F1 score is',f1_round)

#RECALL
recall = recall_score(y_test, y_pred1)
print('Recall score is: ',recall)

print('Precision score is : ',precision_score(y_test,y_pred1))

#plot a confusion matrix
print(confusion_matrix(y_test,y_pred1))


*******************GaussianNB*****************
Accuracy score is 0.8834080717488789
F1 score is 0.69
Recall score is:  0.9042553191489362
Precision score is :  0.551948051948052
[[1012  138]
 [  18  170]]


In [27]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
#Evaluating ,MultinomialNB classifier 
print('*******************MultinomialNB*****************')
print('Accuracy score is',accuracy_score(y_test,y_pred2))

# F1-SCORE
f1=f1_score(y_pred2,y_test)
f1_round=round(f1,2)
print('F1 score is',f1_round)

#RECALL
recall = recall_score(y_test, y_pred2)
print('Recall score is: ',recall)

print('Precision score is : ',precision_score(y_test,y_pred2))

#plot a confusion matrix
print(confusion_matrix(y_test,y_pred2))



*******************MultinomialNB*****************
Accuracy score is 0.9783258594917787
F1 score is 0.92
Recall score is:  0.9148936170212766
Precision score is :  0.9297297297297298
[[1137   13]
 [  16  172]]


In [28]:
bnb.fit(X_train,y_train)
#Evaluating
y_pred3 = bnb.predict(X_test)
print('*******************BernoulliNB*****************')

print('Accuracy score is',accuracy_score(y_test,y_pred3))

# F1-SCORE
f1=f1_score(y_pred3,y_test)
f1_round=round(f1,2)
print('F1 score is',f1_round)

#RECALL
recall = recall_score(y_test, y_pred3)
print('Recall score is: ',recall)

print('Precision score is : ',precision_score(y_test,y_pred3))

#plot a confusion matrix
print(confusion_matrix(y_test,y_pred3))

*******************BernoulliNB*****************
Accuracy score is 0.9596412556053812
F1 score is 0.84
Recall score is:  0.7393617021276596
Precision score is :  0.9652777777777778
[[1145    5]
 [  49  139]]


In [29]:
#Decisin tree
print('*******************Decisin tree*****************')

from sklearn.metrics import confusion_matrix 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
   
# Function to split the dataset 
 
# Function to perform training with giniIndex. 
def train_using_gini(X_train, X_test, y_train): 
   
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", 
            random_state = 100,max_depth=3, min_samples_leaf=5) 
   
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 
       
# Function to perform training with entropy. 
def tarin_using_entropy(X_train, X_test, y_train): 
   
    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 100, 
            max_depth = 3, min_samples_leaf = 5) 
   
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy 
   
# Function to make predictions 
def prediction(X_test, clf_object): 
   
    # Predicton on test with giniIndex 
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:") 
    print(y_pred) 
    return y_pred 
       
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
       
    print("Confusion Matrix: ", 
        confusion_matrix(y_test, y_pred)) 
       
    print ("Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
       
    print("Report : ", 
    classification_report(y_test, y_pred))   
#Driver code 
def main(): 
       
    # Building Phase 
   
    clf_gini = train_using_gini(X_train, X_test, y_train) 
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train) 
       
    # Operational Phase 
    print("Results Using Gini Index:") 
       
    # Prediction using gini 
    y_pred_gini = prediction(X_test, clf_gini) 
    cal_accuracy(y_test, y_pred_gini) 
       
    print("Results Using Entropy:") 
    # Prediction using entropy 
    y_pred_entropy = prediction(X_test, clf_entropy) 
    cal_accuracy(y_test, y_pred_entropy) 
            
# Calling main function 
main()

*******************Decisin tree*****************
Results Using Gini Index:
Predicted values:
[0 0 0 ... 0 0 0]
Confusion Matrix:  [[1118   32]
 [  76  112]]
Accuracy :  91.92825112107623
Report :                precision    recall  f1-score   support

           0       0.94      0.97      0.95      1150
           1       0.78      0.60      0.67       188

    accuracy                           0.92      1338
   macro avg       0.86      0.78      0.81      1338
weighted avg       0.91      0.92      0.91      1338

Results Using Entropy:
Predicted values:
[0 0 0 ... 0 0 0]
Confusion Matrix:  [[1108   42]
 [  65  123]]
Accuracy :  92.00298953662183
Report :                precision    recall  f1-score   support

           0       0.94      0.96      0.95      1150
           1       0.75      0.65      0.70       188

    accuracy                           0.92      1338
   macro avg       0.85      0.81      0.83      1338
weighted avg       0.92      0.92      0.92      1338

